# Cargando WDB en MongoDB

In [1]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import csv

In [2]:
# connect to database
connection = MongoClient('localhost', 27017)

In [3]:
# Borramos la base de datos
connection.drop_database('SWP')

# Pymongo permite una sintaxis practicamente igual que la original
dbCnxSWP = connection.SWP

### Preparamos el fichero para carga

In [4]:
!cat /home/learner/notebooks/data/WDB_Accused.txt      \
    | perl -pe 's/^(\r\n)//g'                          \
    | perl -pe 's/([^:]..)\r\n/\1\\r\\n/g'             \
    | sed -e ':a;s/^\(\("[^"]*"\|[^",]*\)*\),/\1|/;ta' \
    | sed 's/"//g'                                   \
    > /home/learner/notebooks/data/WDB_Accused_loadMongoDB.txt

#    | perl -pe 's/^(\r\n)//g' \                          # Delete lines whith return alone
#    | perl -pe 's/([^:]..)\r\n/\1\\r\\n/g' \             # Delete return whith third char to the left not :
#    | sed -e ':a;s/^\(\("[^"]*"\|[^",]*\)*\),/\1|/;ta' \ # Replace delimiter , with | (not between "")
#    | sed -e '/|$/d' \                                   # Filter line with Age not informed

!head -3 /home/learner/notebooks/data/WDB_Accused_loadMongoDB.txt
!wc -l /home/learner/notebooks/data/WDB_Accused_loadMongoDB.txt

A/EGD/10|EGD|10|Mareon|Quheitt|Marion|White||||Female||0|0|Sammuelston|P/JO/3539|Haddington|Haddington||||||||||SMD|15/5/2001 11:06:51|jhm|9/8/2002 11:40:51
A/EGD/100|EGD|100|Thom|Cockburn|Thomas|Cockburn||||Male||0|0||||Haddington||||||||||SMD|15/5/2001 11:06:51|jhm|2/10/2002 10:32:30
A/EGD/1000|EGD|1000|Christian|Aitkenhead|Christine|Aikenhead||||Female||0|0|Rottinraw|||Dumfries||||||Married||||SMD|15/5/2001 11:06:51|jhm|1/10/2002 10:48:12
3219 /home/learner/notebooks/data/WDB_Accused_loadMongoDB.txt


In [5]:
WDB_Accused_data_path = '/home/learner/notebooks/data/WDB_Accused_loadMongoDB.txt'
WDB_Accused_csvFile = open(WDB_Accused_data_path, "r")
iMax = 5
i = 0
for WDB_Accused_csvLine in WDB_Accused_csvFile:
    print "\rWDB_Accused_csvLine:", WDB_Accused_csvLine,
    i = i + 1
    if i > iMax:
        break
WDB_Accused_csvFile.close()

WDB_Accused_csvLine: A/EGD/10|EGD|10|Mareon|Quheitt|Marion|White||||Female||0|0|Sammuelston|P/JO/3539|Haddington|Haddington||||||||||SMD|15/5/2001 11:06:51|jhm|9/8/2002 11:40:51
WDB_Accused_csvLine: A/EGD/100|EGD|100|Thom|Cockburn|Thomas|Cockburn||||Male||0|0||||Haddington||||||||||SMD|15/5/2001 11:06:51|jhm|2/10/2002 10:32:30
WDB_Accused_csvLine: A/EGD/1000|EGD|1000|Christian|Aitkenhead|Christine|Aikenhead||||Female||0|0|Rottinraw|||Dumfries||||||Married||||SMD|15/5/2001 11:06:51|jhm|1/10/2002 10:48:12
WDB_Accused_csvLine: A/EGD/1001|EGD|1001|Janet|Ireland|Janet|Ireland||||Female||0|0|Rottinraw|||Dumfries||||||Widowed||||SMD|15/5/2001 11:06:51|jhm|1/10/2002 10:49:00
WDB_Accused_csvLine: A/EGD/1002|EGD|1002|Agnes|Hendersoun|Agnes|Henderson||||Female||0|0||P/ST/1446|Stirling|Stirling||||||||||SMD|15/5/2001 11:06:51|jhm|1/10/2002 10:50:07
WDB_Accused_csvLine: A/EGD/1003|EGD|1003|Margaret|Young|Margaret|Young||||Female||0|0||P/ST/1359|Haddington|Haddington||||||||||SMD|15/5/2001 11:06:51|

In [6]:
print dbCnxSWP.collection_names()

print dbCnxSWP.WDB_Accused.count()


[]
0


In [7]:
from pymongo import InsertOne, DeleteOne, ReplaceOne

# Eliminamos antes resultados de cargas anteriores
result = dbCnxSWP.WDB_Accused.delete_many({})
print result.deleted_count

WDB_Accused_csvFile = open(WDB_Accused_data_path, "r")

iMax = 5000
i = 1

for WDB_Accused_csvLine in WDB_Accused_csvFile:

    if i > iMax:
        break

    i = i + 1 

    result = dbCnxSWP.WDB_Accused.insert_one(
            {
                "_id"       		: WDB_Accused_csvLine.split("|")[0],
                "AccusedRef"		: WDB_Accused_csvLine.split("|")[0],
                "AccusedSystemId"	: WDB_Accused_csvLine.split("|")[1],
                "AccusedID"			: WDB_Accused_csvLine.split("|")[2],
                "FirstName"			: WDB_Accused_csvLine.split("|")[3],
                "LastName"			: WDB_Accused_csvLine.split("|")[4],
                "M_Firstname"		: WDB_Accused_csvLine.split("|")[5],
                "M_Surname"			: WDB_Accused_csvLine.split("|")[6],
                "Alias"				: WDB_Accused_csvLine.split("|")[7],
                "Patronymic"		: WDB_Accused_csvLine.split("|")[8],
                "DesTitle"			: WDB_Accused_csvLine.split("|")[9],
                "Sex"				: WDB_Accused_csvLine.split("|")[10],
                "Age"				: WDB_Accused_csvLine.split("|")[11],
                "Age_estcareer"		: WDB_Accused_csvLine.split("|")[12],
                "Age_estchild"		: WDB_Accused_csvLine.split("|")[13],
                "Res_settlement"	: WDB_Accused_csvLine.split("|")[14],
                "Res_parish"		: WDB_Accused_csvLine.split("|")[15],
                "Res_presbytery"	: WDB_Accused_csvLine.split("|")[16],
                "Res_county"		: WDB_Accused_csvLine.split("|")[17],
                "Res_burgh"			: WDB_Accused_csvLine.split("|")[18],
                "Res_NGR_Letters"	: WDB_Accused_csvLine.split("|")[19],
                "Res_NGR_Easting"	: WDB_Accused_csvLine.split("|")[20],
                "Res_NGR_Northing"	: WDB_Accused_csvLine.split("|")[21],
                "Ethnic_origin"		: WDB_Accused_csvLine.split("|")[22],
                "MaritalStatus"		: WDB_Accused_csvLine.split("|")[23],
                "SocioecStatus"		: WDB_Accused_csvLine.split("|")[24],
                "Occupation"		: WDB_Accused_csvLine.split("|")[25],
                "Notes"				: WDB_Accused_csvLine.split("|")[26],
                "Createdby"			: WDB_Accused_csvLine.split("|")[27],
                "Createdate"		: WDB_Accused_csvLine.split("|")[28],
                "Lastupdatedby"		: WDB_Accused_csvLine.split("|")[29],
                "Lastupdatedon"		: WDB_Accused_csvLine.split("|")[30],
            }
        )
    print "\rresult.inserted_id:", result.inserted_id, i, 
    
    
WDB_Accused_csvFile.close()


0
result.inserted_id: A/LA/3244 3220


In [8]:
dbCnxSWP.WDB_Accused.drop_indexes()

In [9]:
dbCnxSWP.WDB_Accused.create_index("AccusedRef", unique=True)

u'AccusedRef_1'

In [10]:
for index in dbCnxSWP.WDB_Accused.list_indexes():
    print index

SON([(u'v', 2), (u'key', SON([(u'_id', 1)])), (u'name', u'_id_'), (u'ns', u'SWP.WDB_Accused')])
SON([(u'v', 2), (u'unique', True), (u'key', SON([(u'AccusedRef', 1)])), (u'name', u'AccusedRef_1'), (u'ns', u'SWP.WDB_Accused')])


In [11]:
dbCnxSWP.WDB_Accused.index_information()

{u'AccusedRef_1': {u'key': [(u'AccusedRef', 1)],
  u'ns': u'SWP.WDB_Accused',
  u'unique': True,
  u'v': 2},
 u'_id_': {u'key': [(u'_id', 1)], u'ns': u'SWP.WDB_Accused', u'v': 2}}

## Contamos el número de registros cargados

In [12]:
print "Número de WDB_Accused: " , dbCnxSWP.WDB_Accused.count()

Número de WDB_Accused:  3219


### Preparamos fichero WDB_Accused_family para la carga

In [13]:
!cat /home/learner/notebooks/data/WDB_Accused_family.txt      \
    | perl -pe 's/^(\r\n)//g'                          \
    | perl -pe 's/([^:]..)\r\n/\1\\r\\n/g'             \
    | sed -e ':a;s/^\(\("[^"]*"\|[^",]*\)*\),/\1|/;ta' \
    | sed 's/"//g'                                   \
    > /home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt

!head -3 /home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt
!wc -l /home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt

AF/JO/1|JO|1|Leyis|John||||||0|0|stabler|husband|A/EGD/2067|jhm|16/5/2001 14:45:56|jhm|16/5/2001 14:48:31
AF/JO/10|JO|10|Wischert|Jonet||||||0|0||Mother|A/EGD/2068|jhm|17/5/2001 16:24:38|jhm|17/5/2001 16:25:23
AF/JO/100|JO|100|Smiberd|Thomas|||||50|0|1||Father|A/EGD/1022|jhm|3/7/2001 09:14:26|jhm|3/7/2001 09:14:39
951 /home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt


In [14]:
# Ensayo de lectura del fichero identificando la clave
# que nos servirá para localizar el documento donde tenemos que incluir cada registro

WDB_Accused_family_data_path = '/home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt'
WDB_Accused_family_csvFile = open(WDB_Accused_family_data_path, "r")

iMax = 5
i = 0
for WDB_Accused_family_csvLine in WDB_Accused_family_csvFile:
    print "\rWDB_Accused_family_csvLine:", WDB_Accused_family_csvLine,
    print WDB_Accused_family_csvLine.split("|")[14]
    i = i + 1
    if i > iMax:
        break
WDB_Accused_family_csvFile.close()

WDB_Accused_family_csvLine: AF/JO/1|JO|1|Leyis|John||||||0|0|stabler|husband|A/EGD/2067|jhm|16/5/2001 14:45:56|jhm|16/5/2001 14:48:31
A/EGD/2067
WDB_Accused_family_csvLine: AF/JO/10|JO|10|Wischert|Jonet||||||0|0||Mother|A/EGD/2068|jhm|17/5/2001 16:24:38|jhm|17/5/2001 16:25:23
A/EGD/2068
WDB_Accused_family_csvLine: AF/JO/100|JO|100|Smiberd|Thomas|||||50|0|1||Father|A/EGD/1022|jhm|3/7/2001 09:14:26|jhm|3/7/2001 09:14:39
A/EGD/1022
WDB_Accused_family_csvLine: AF/JO/101|JO|101|Smiberd|Thomas|||||50|0|1||Husband|A/EGD/1023|jhm|3/7/2001 09:33:20|jhm|3/7/2001 09:33:37
A/EGD/1023
WDB_Accused_family_csvLine: AF/JO/102|JO|102|Smiberd|Jenet|||||25|1|0||Daughter|A/EGD/1023|jhm|3/7/2001 09:33:39|jhm|3/7/2001 09:33:59
A/EGD/1023
WDB_Accused_family_csvLine: AF/JO/103|JO|103|Phenick|John|||||50|0|1|Tailor|Father|A/EGD/1204|jhm|4/7/2001 10:42:22|jhm|4/7/2001 10:43:16
A/EGD/1204


In [15]:
from pymongo import InsertOne, DeleteOne, ReplaceOne, UpdateOne

iMax = 3
i = 0
nmods = 0
nnomods = 0

WDB_Accused_family_data_path = '/home/learner/notebooks/data/WDB_Accused_family_loadMongoDB.txt'
WDB_Accused_family_csvFile = open(WDB_Accused_family_data_path, "r")

for WDB_Accused_family_csvLine in WDB_Accused_family_csvFile:

#    if i > iMax:
#        break

    i = i + 1 

    result = dbCnxSWP.WDB_Accused.update_one(
        { "_id": WDB_Accused_family_csvLine.split("|")[14]},
        { '$push': { "Accused_family" : {
                                        "Accused_familyRef"		: WDB_Accused_family_csvLine.split("|")[0],
                                        "Accused_familySystemId": WDB_Accused_family_csvLine.split("|")[1],
                                        "Accused_familyID"		: WDB_Accused_family_csvLine.split("|")[2],
                                        "Surname"				: WDB_Accused_family_csvLine.split("|")[3],
                                        "Firstname"				: WDB_Accused_family_csvLine.split("|")[4],
                                        "Alias"					: WDB_Accused_family_csvLine.split("|")[5],
                                        "Patronymic"			: WDB_Accused_family_csvLine.split("|")[6],
                                        "DesTitle"				: WDB_Accused_family_csvLine.split("|")[7],
                                        "Est_Year_of_Birth"		: WDB_Accused_family_csvLine.split("|")[8],
                                        "Age"					: WDB_Accused_family_csvLine.split("|")[9],
                                        "Age_estcareer"			: WDB_Accused_family_csvLine.split("|")[10],
                                        "Age_estchild"			: WDB_Accused_family_csvLine.split("|")[11],
                                        "Occupation"			: WDB_Accused_family_csvLine.split("|")[12],
                                        "Relationship"			: WDB_Accused_family_csvLine.split("|")[13],
                                        "AccusedRef"			: WDB_Accused_family_csvLine.split("|")[14],
                                        "Createdby"				: WDB_Accused_family_csvLine.split("|")[15],
                                        "Createdate"			: WDB_Accused_family_csvLine.split("|")[16],
                                        "Lastupdatedby"			: WDB_Accused_family_csvLine.split("|")[17],
                                        "Lastupdatedon"			: WDB_Accused_family_csvLine.split("|")[18]
                                        }
                } 
        }
        );

    if result.modified_count == 1:
        nmods = nmods + 1
    else:
        nnomods = nnomods + 1
    
    print "\r", i, "Modificados:", nmods, " No modificados:", nnomods,
    

WDB_Accused_family_csvFile.close()


951 Modificados: 951  No modificados: 0


In [16]:
# Recuperamos uno de los registros cargados
for WDB_Accused in dbCnxSWP.WDB_Accused.find({"_id":"A/EGD/143"}):
#    print "WDB_Accused:",WDB_Accused
#    print "WDB_Accused[FirstName]:",WDB_Accused["FirstName"]
#    print "WDB_Accused[Accused_family]:",WDB_Accused["Accused_family"]
    print "WDB_Accused[Accused_family][0]:",WDB_Accused["Accused_family"][0]
    print "WDB_Accused[Accused_family][5]:",WDB_Accused["Accused_family"][5]
    

WDB_Accused[Accused_family][0]: {u'AccusedRef': u'A/EGD/143', u'Lastupdatedon': u'8/10/2002 14:41:09\r\n', u'Surname': u'Erskine', u'Relationship': u'Nephew', u'Firstname': u'Johnne', u'Age_estcareer': u'0', u'Age_estchild': u'0', u'Createdate': u'8/10/2002 14:41:03', u'Accused_familySystemId': u'JO', u'Est_Year_of_Birth': u'', u'Accused_familyID': u'698', u'Alias': u'', u'DesTitle': u'', u'Lastupdatedby': u'jhm', u'Createdby': u'jhm', u'Accused_familyRef': u'AF/JO/698', u'Patronymic': u'', u'Age': u'', u'Occupation': u''}
WDB_Accused[Accused_family][5]: {u'AccusedRef': u'A/EGD/143', u'Lastupdatedon': u'26/6/2001 12:19:15\r\n', u'Surname': u'Erskine', u'Relationship': u'Father', u'Firstname': u'Johnne', u'Age_estcareer': u'0', u'Age_estchild': u'0', u'Createdate': u'26/6/2001 12:18:54', u'Accused_familySystemId': u'LA', u'Est_Year_of_Birth': u'', u'Accused_familyID': u'83', u'Alias': u'', u'DesTitle': u'', u'Lastupdatedby': u'LEM', u'Createdby': u'LEM', u'Accused_familyRef': u'AF/LA/83

In [17]:
dbCnxSWP.WDB_Accused.Accused_family.drop_indexes()

In [18]:
dbCnxSWP.WDB_Accused.Accused_family.create_index("Accused_family.Accused_familyRef", unique=True, sparse=True  )

u'Accused_family.Accused_familyRef_1'

In [19]:
dbCnxSWP.WDB_Accused.Accused_family.index_information()

{u'Accused_family.Accused_familyRef_1': {u'key': [(u'Accused_family.Accused_familyRef',
    1)],
  u'ns': u'SWP.WDB_Accused.Accused_family',
  u'sparse': True,
  u'unique': True,
  u'v': 2},
 u'_id_': {u'key': [(u'_id', 1)],
  u'ns': u'SWP.WDB_Accused.Accused_family',
  u'v': 2}}

### Duda: Con estos índices no he conseguido que falle la inserción del subdocumento con los familiares cuando se realiza dos veces. Se duplican los registros.

# Comenzamos a hacernos preguntas interesantes
## ¿Cuántos familiares tenían los acusados?

In [20]:
cursor = dbCnxSWP.WDB_Accused.aggregate( [ { "$unwind": "$Accused_family" }
                                         , { "$group" : { "_id" : "$AccusedRef"
                                                        ,"relatives": { "$sum" : 1 }
                                                         }
                                            }
                                         , { "$group" : { "_id" : { "relatives" :"$relatives" }
                                                        ,"Accuseds": { "$sum" : 1 }
                                                         }
                                            }
                                         , { "$sort"  : { "Accuseds" : -1 } }
                                        ] )

for item in cursor:
    print item

{u'_id': {u'relatives': 1}, u'Accuseds': 554}
{u'_id': {u'relatives': 2}, u'Accuseds': 93}
{u'_id': {u'relatives': 3}, u'Accuseds': 35}
{u'_id': {u'relatives': 5}, u'Accuseds': 10}
{u'_id': {u'relatives': 4}, u'Accuseds': 8}
{u'_id': {u'relatives': 6}, u'Accuseds': 4}


## ¿Qué acusados tenían más familiares?

In [21]:
cursor = dbCnxSWP.WDB_Accused.aggregate( [ { "$unwind": "$Accused_family" }
                                         , { "$group" : { "_id" : { "AccusedRef" :  "$AccusedRef"
                                                                   , "FirstName" : "$FirstName"
                                                                   , "LastName"  : "$LastName"
                                                                  }
                                                        ,"relatives": { "$sum" : 1 }
                                                         }
                                            }
                                         , { "$sort"  : { "relatives" : -1 } }
                                         , { "$limit" : 10 }
                                        ] )

print "Acusados con más familiares registrados:"
for item in cursor:
    print item["_id"]["FirstName"], item["_id"]["LastName"]+":", item["relatives"], "familiares."

Acusados con más familiares registrados:
Issobell Erskine: 6 familiares.
Annas Erskine: 6 familiares.
Margret Jackson: 6 familiares.
Katherine Wilson: 6 familiares.
Annabell Stewart: 5 familiares.
Jonet Mathie: 5 familiares.
Johnnet Wischert: 5 familiares.
Violat Leyis: 5 familiares.
Jonet Leyis: 5 familiares.
Elspet Leyis: 5 familiares.


### ¿Quienes eran sus familiares y que relación tenían?

In [22]:
csrAccused = dbCnxSWP.WDB_Accused.aggregate( [ { "$unwind": "$Accused_family" }
                                         , { "$group" : { "_id" : { "AccusedRef" :  "$AccusedRef"
                                                                   , "FirstName" : "$FirstName"
                                                                   , "LastName"  : "$LastName"
                                                                  }
                                                        ,"relatives": { "$sum" : 1 }
                                                         }
                                            }
                                         , { "$sort"  : { "relatives" : -1 } }
                                         , { "$limit" : 10 }
                                        ] )

for accusedAgg in csrAccused:
    for accused in dbCnxSWP.WDB_Accused.find({"_id":accusedAgg["_id"]["AccusedRef"]}):
        print "\nRelatives of", accused["FirstName"], accused["LastName"] + ':'
        for accused_family in accused["Accused_family"]:
            print "\t*", accused_family["Relationship"]+":", accused_family["Firstname"], accused_family["Surname"]


Relatives of Issobell Erskine:
	* Nephew: Johnne Erskine
	* Nephew: Alexander Erskine
	* Brother: Robert Erskine
	* Sister: Annas Erskine
	* Sister: Helene Erskine
	* Father: Johnne Erskine

Relatives of Annas Erskine:
	* Nephew: Johnne Erskine
	* Nephew: Alexnder Erskine
	* Brother: Robert Erskine
	* Sister: Helene Erskine
	* Sister: Issobell Erskine
	* Father: Johnne Erskine

Relatives of Margret Jackson:
	* Husband: Thomas Stewart
	* In-law: Annabell Stewart
	* In-law: John Stewart
	* In-law: Hugh Stewart
	* In-law: Jonet Mathie
	* In-law: John Stewart

Relatives of Katherine Wilson:
	* Son: James Ruchheid
	* Daughter: Margaret Ruchheid
	* Husband: Thomas Ruchheid
	* Sister: Margaret Wilson
	* Son-in-law: John Home
	* Brother-in-law: Johne Prat

Relatives of Annabell Stewart:
	* Father: Jon Stewart
	* Brother: Jon Stewart
	* Mother: Jonet Mathie
	* Brother: Hugh Stewart
	* In-law: Margaret Jackson

Relatives of Jonet Mathie:
	* Husband: Jon Stewart
	* Son: Jon Stewart
	* Daughter: 

### Un ejemplo de uso de información no estructurada. El clásico conteo de palabras con map-reduce.
#### Veamos cuales fueron los *trending topics* en los comentarios respecto a los acusados

In [23]:
from bson.code import Code
mapper = Code("""
               function () {
                 this.Notes.split(" ").forEach(function(z) {
                   emit(z, 1);
                 });
               }
               """)

In [24]:
reducer = Code("""
                function (key, values) {
                  var total = 0;
                  for (var i = 0; i < values.length; i++) {
                    total += values[i];
                  }
                  return total;
                }
                """)

In [25]:
result = dbCnxSWP.WDB_Accused.map_reduce(mapper, reducer, "myresults")

min_value = 32

stopwords_data_path = '/home/learner/notebooks/data/stop_words_english.txt'
stopwords_csvFile = open(stopwords_data_path, "r")

stopwords_list = stopwords_csvFile.read().splitlines()
#print stopwords_list 

for doc in result.find({ "_id" : { "$nin" : stopwords_list }}).sort("value",-1):
    print (doc)
    if doc["value"] < min_value:
        break

stopwords_csvFile.close()

{u'_id': u'', u'value': 2307.0}
{u'_id': u'She', u'value': 126.0}
{u'_id': u'also', u'value': 126.0}
{u'_id': u'recorded', u'value': 116.0}
{u'_id': u'accused', u'value': 104.0}
{u'_id': u'details.', u'value': 86.0}
{u'_id': u'I', u'value': 85.0}
{u'_id': u'details', u'value': 76.0}
{u'_id': u'Her', u'value': 72.0}
{u'_id': u'indweller', u'value': 71.0}
{u'_id': u'parish', u'value': 68.0}
{u'_id': u'Recorded', u'value': 62.0}
{u'_id': u'husband', u'value': 60.0}
{u'_id': u'presbytery', u'value': 59.0}
{u'_id': u'years', u'value': 59.0}
{u'_id': u'Husband', u'value': 58.0}
{u'_id': u'No', u'value': 55.0}
{u'_id': u'The', u'value': 54.0}
{u'_id': u'may', u'value': 54.0}
{u'_id': u'name', u'value': 52.0}
{u'_id': u'described', u'value': 50.0}
{u'_id': u'accused.', u'value': 45.0}
{u'_id': u'modernise', u'value': 43.0}
{u'_id': u'He', u'value': 41.0}
{u'_id': u'person', u'value': 41.0}
{u'_id': u'think', u'value': 41.0}
{u'_id': u'Unable', u'value': 38.0}
{u'_id': u'witchcraft.', u'value':

#### Destacamos los términos más llamativos.
{u'_id': u'indweller', u'value': 71.0} (Posesión)  
{u'_id': u'parish', u'value': 68.0} (Parroquia)  
{u'_id': u'Recorded', u'value': 62.0} (Registrado)  
{u'_id': u'husband', u'value': 60.0} (Marido)  
{u'_id': u'presbytery', u'value': 59.0}(Distrito eclesiastico)  
{u'_id': u'probably', u'value': 33.0}(Probablemente)  
{u'_id': u'wife', u'value': 31.0} (Mujer)  
{u'_id': u'witch', u'value': 31.0} (Bruja)  
{u'_id': u'mentioned', u'value': 20.0} (Referido)  
{u'_id': u'Possibly', u'value': 19.0} (Posiblemente)


## Edades más comunes

In [26]:
cursor = dbCnxSWP.WDB_Accused.aggregate([
        {"$group": {"_id": "$Age", "count": {"$sum": 1}}},
        {"$sort" : {"count" : -1}},
        {"$limit" : 10}
    ])

for row in cursor:
    print row["_id"],"-", row["count"]

 - 3053
50 - 36
45 - 12
55 - 7
25 - 7
60 - 6
39 - 6
43 - 5
41 - 5
30 - 5


### Preparamos fichero WDB_Case para la carga

In [27]:
!cat /home/learner/notebooks/data/WDB_Case.txt      \
    | perl -pe 's/^(\r\n)//g'                          \
    | perl -pe 's/([^:]..)\r\n/\1\\r\\n/g'             \
    | sed -e ':a;s/^\(\("[^"]*"\|[^",]*\)*\),/\1|/;ta' \
    | sed 's/"//g'                                   \
    > /home/learner/notebooks/data/WDB_Case_loadMongoDB.txt

In [28]:
!head -3 /home/learner/notebooks/data/WDB_Case_loadMongoDB.txt
print "\nRegistros para cargar: ", 
!wc -l /home/learner/notebooks/data/WDB_Case_loadMongoDB.txt | cut -f1 -d " "

C/EGD/10|10|EGD|0||2/8/1649|2/8/1649 00:00:00|2/8/1649|2/8/1649 00:00:00||3 unnamed witches|commission for execution|||||3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0||1|1|0|0||0||0||0|0|0|0|0|0|0||||0|0|0|0|0|0|0|0|0|0||0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0||0|0|||0|0|0|0|0||certain persons from Haymouth.|SMD|15/5/2001 17:00:16|jhm|30/7/2002 15:32:43
C/EGD/100|100|EGD|-1|A/EGD/89|27/1/1591|27/1/1591 00:00:00|8/5/1591|8/5/1591 00:00:00||Kerington,Cristine|||||||0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0||1|1|0|0|Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, second to trouble King James after he went to get her in Denmark.|0|Said by others to have met the devil as a 'mekle black man'|0||0|0|0|0|0|0|0|||See Agnes Sampson's case for description of North Berwick meeting (c/egd/63).  See also Napi

In [29]:
# Ensayo de lectura del fichero identificando la clave que nos servirá
# para localizar el documento donde tenemos que incluir cada registro

WDB_Case_data_path = '/home/learner/notebooks/data/WDB_Case_loadMongoDB.txt'
WDB_Case_csvFile = open(WDB_Case_data_path, "r")

iMax = 5
i = 0
for WDB_Case_csvLine in WDB_Case_csvFile:
    print "AccusedRef: ", WDB_Case_csvLine.split("|")[4],"\nWDB_Case_csvLine:", WDB_Case_csvLine, "\n"
    print WDB_Case_csvLine.split("|")[110]
    i = i + 1
    if i > iMax:
        break
WDB_Case_csvFile.close()

AccusedRef:   
WDB_Case_csvLine: C/EGD/10|10|EGD|0||2/8/1649|2/8/1649 00:00:00|2/8/1649|2/8/1649 00:00:00||3 unnamed witches|commission for execution|||||3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0||1|1|0|0||0||0||0|0|0|0|0|0|0||||0|0|0|0|0|0|0|0|0|0||0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0||0|0|||0|0|0|0|0||certain persons from Haymouth.|SMD|15/5/2001 17:00:16|jhm|30/7/2002 15:32:43


30/7/2002 15:32:43

AccusedRef:  A/EGD/89 
WDB_Case_csvLine: C/EGD/100|100|EGD|-1|A/EGD/89|27/1/1591|27/1/1591 00:00:00|8/5/1591|8/5/1591 00:00:00||Kerington,Cristine|||||||0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0||1|1|0|0|Part of the North Berwick witch hunt.  It involved many people and many accustions about witches meetings, specifically two separate attempts to sink the royal ships.  The first to sink Queen Anne on her way home, second to trouble King James after he went to get her in Denmark.|0|Said by others to have met the devil as a 'mekle black man'|0||0|0|0|0|0|0|

In [30]:
from pymongo import InsertOne, DeleteOne, ReplaceOne, UpdateOne

iMax = 3500 # > 3413 para cargar todos los registros
i = 0
nmods = 0
nnomods = 0

WDB_Case_data_path = '/home/learner/notebooks/data/WDB_Case_loadMongoDB.txt'
WDB_Case_csvFile = open(WDB_Case_data_path, "r")

for WDB_Case_csvLine in WDB_Case_csvFile:

    if i > iMax:
        break

    i = i + 1 

       
    result = dbCnxSWP.WDB_Accused.update_one(
        { "_id": WDB_Case_csvLine.split("|")[4]},
        { '$push': { "Case" : {         "CaseRef"		: WDB_Case_csvLine.split("|")[0],
                                        "CaseId"		: WDB_Case_csvLine.split("|")[1],
                                        "CaseSystemId"		: WDB_Case_csvLine.split("|")[2],
                                        "NamedIndividual"	: WDB_Case_csvLine.split("|")[3],
                                        "AccusedRef"		: WDB_Case_csvLine.split("|")[4],
                                        "CaseStart_date"	: WDB_Case_csvLine.split("|")[5],
                                        "CaseStart_date_as_date"	: WDB_Case_csvLine.split("|")[6],
                                        "Case_date"					: WDB_Case_csvLine.split("|")[7],
                                        "Case_date_as_date"		: WDB_Case_csvLine.split("|")[8],
                                        "Age_at_case"			: WDB_Case_csvLine.split("|")[9],
                                        "CaseCommonName"	: WDB_Case_csvLine.split("|")[10],
                                        "Commission"		: WDB_Case_csvLine.split("|")[11],
                                        "Complaint"			: WDB_Case_csvLine.split("|")[12],
                                        "Correspondence"	: WDB_Case_csvLine.split("|")[13],
                                        "Chronicle"			: WDB_Case_csvLine.split("|")[14],
                                        "Other"				: WDB_Case_csvLine.split("|")[15],
                                        "Suspects_text"			: WDB_Case_csvLine.split("|")[16],
                                        "UNorthodoxRelPract_p"	: WDB_Case_csvLine.split("|")[17],
                                        "UNorthodoxRelPract_s"	: WDB_Case_csvLine.split("|")[18],
                                        "Consulting_p"			: WDB_Case_csvLine.split("|")[19],
                                        "Consulting_s"			: WDB_Case_csvLine.split("|")[20],
                                        "Demonic_p"			: WDB_Case_csvLine.split("|")[21],
                                        "Demonic_s"			: WDB_Case_csvLine.split("|")[22],
                                        "Demonic_possess_p"	: WDB_Case_csvLine.split("|")[23],
                                        "Demonic_possess_s"	: WDB_Case_csvLine.split("|")[24],
                                        "Fairies_p"			: WDB_Case_csvLine.split("|")[25],
                                        "Fairies_s"			: WDB_Case_csvLine.split("|")[26],
                                        "Folk_healing_p"	: WDB_Case_csvLine.split("|")[27],
                                        "Folk_healing_s"	: WDB_Case_csvLine.split("|")[28],
                                        "Maleficium_p"		: WDB_Case_csvLine.split("|")[29],
                                        "Maleficium_s"	: WDB_Case_csvLine.split("|")[30],
                                        "Midwifery_p"	: WDB_Case_csvLine.split("|")[31],
                                        "Midwifery_s"			: WDB_Case_csvLine.split("|")[32],
                                        "ImplicatedByAnother_p"	: WDB_Case_csvLine.split("|")[33],
                                        "ImplicatedByAnother_s"	: WDB_Case_csvLine.split("|")[34],
                                        "Neighbhd_dispute_p"	: WDB_Case_csvLine.split("|")[35],
                                        "Neighbhd_dispute_s"	: WDB_Case_csvLine.split("|")[36],
                                        "PoliticalMotive_p"		: WDB_Case_csvLine.split("|")[37],
                                        "PoliticalMotive_s"	: WDB_Case_csvLine.split("|")[38],
                                        "PropertyMotive_p"	: WDB_Case_csvLine.split("|")[39],
                                        "PropertyMotive_s"	: WDB_Case_csvLine.split("|")[40],
                                        "RefusedCharity_p"	: WDB_Case_csvLine.split("|")[41],
                                        "RefusedCharity_s"	: WDB_Case_csvLine.split("|")[42],
                                        "Treason_p"			: WDB_Case_csvLine.split("|")[43],
                                        "Treason_s"			: WDB_Case_csvLine.split("|")[44],
                                        "Other_p"			: WDB_Case_csvLine.split("|")[45],
                                        "Other_s"			: WDB_Case_csvLine.split("|")[46],
                                        "OtherText" 		: WDB_Case_csvLine.split("|")[47],
                                        "NotEnoughInfo_p"	: WDB_Case_csvLine.split("|")[48],
                                        "NotEnoughInfo_s"	: WDB_Case_csvLine.split("|")[49],
                                        "WhiteMagic_p"		: WDB_Case_csvLine.split("|")[50],
                                        "WhiteMagic_s"	: WDB_Case_csvLine.split("|")[51],
                                        "Charnotes"		: WDB_Case_csvLine.split("|")[52],
                                        "DemonicPact"	: WDB_Case_csvLine.split("|")[53],
                                        "DevilNotes"		: WDB_Case_csvLine.split("|")[54],
                                        "WitchesMeeting"	: WDB_Case_csvLine.split("|")[55],
                                        "MeetingName"		: WDB_Case_csvLine.split("|")[56],
                                        "DevilPresent"		: WDB_Case_csvLine.split("|")[57],
                                        "Maleficium"	: WDB_Case_csvLine.split("|")[58],
                                        "CommunalSex"	: WDB_Case_csvLine.split("|")[59],
                                        "DevilWorship"	: WDB_Case_csvLine.split("|")[60],
                                        "FoodAndDrink"	: WDB_Case_csvLine.split("|")[61],
                                        "Dancing"		: WDB_Case_csvLine.split("|")[62],
                                        "Singing"		: WDB_Case_csvLine.split("|")[63],
                                        "SingingText" 		: WDB_Case_csvLine.split("|")[64],
                                        "OtherPractices" 	: WDB_Case_csvLine.split("|")[65],
                                        "MeetingNotes"		: WDB_Case_csvLine.split("|")[66],
                                        "Elphane/Fairyland"	: WDB_Case_csvLine.split("|")[67],
                                        "Food/Drink"				: WDB_Case_csvLine.split("|")[68],
                                        "SpecificVerbalFormulae"	: WDB_Case_csvLine.split("|")[69],
                                        "SpecificRitualActs"		: WDB_Case_csvLine.split("|")[70],
                                        "Familiars"					: WDB_Case_csvLine.split("|")[71],
                                        "Shape-Changing"			: WDB_Case_csvLine.split("|")[72],
                                        "Dreams/Visions"				: WDB_Case_csvLine.split("|")[73],
                                        "UnorthodoxReligiousPractice"	: WDB_Case_csvLine.split("|")[74],
                                        "SympatheticMagic"				: WDB_Case_csvLine.split("|")[75],
                                        "Ridingdead"		: WDB_Case_csvLine.split("|")[76],
                                        "FolkNotes"			: WDB_Case_csvLine.split("|")[77],
                                        "HumanIllness"		: WDB_Case_csvLine.split("|")[78],
                                        "HumanDeath"		: WDB_Case_csvLine.split("|")[79],
                                        "AnimalIllness"		: WDB_Case_csvLine.split("|")[80],
                                        "AnimalDeath"		: WDB_Case_csvLine.split("|")[81],
                                        "FemaleInfertility"	: WDB_Case_csvLine.split("|")[82],
                                        "MaleImpotence"		: WDB_Case_csvLine.split("|")[83],
                                        "AggravatingDisease"	: WDB_Case_csvLine.split("|")[84],
                                        "TransferringDisease"	: WDB_Case_csvLine.split("|")[85],
                                        "LayingOn"				: WDB_Case_csvLine.split("|")[86],
                                        "Removalbewitchment"	: WDB_Case_csvLine.split("|")[87],
                                        "Quarreling"		: WDB_Case_csvLine.split("|")[88],
                                        "Cursing"			: WDB_Case_csvLine.split("|")[89],
                                        "Poisoning"			: WDB_Case_csvLine.split("|")[90],
                                        "RecHealer"			: WDB_Case_csvLine.split("|")[91],
                                        "HealingHumans"		: WDB_Case_csvLine.split("|")[92],
                                        "HealingAnimals"	: WDB_Case_csvLine.split("|")[93],
                                        "Midwifery"			: WDB_Case_csvLine.split("|")[94],
                                        "DiseaseNotes" 		: WDB_Case_csvLine.split("|")[95],
                                        "PropertyDamage"		: WDB_Case_csvLine.split("|")[96],
                                        "WeatherModification"	: WDB_Case_csvLine.split("|")[97],
                                        "OtherMaleficiaNotes" 	: WDB_Case_csvLine.split("|")[98],
                                        "OtherChargesNotes" 	: WDB_Case_csvLine.split("|")[99],
                                        "ClaimedBewitched"		: WDB_Case_csvLine.split("|")[100],
                                        "ClaimedPossessed"		: WDB_Case_csvLine.split("|")[101],
                                        "AdmitLesserCharge"		: WDB_Case_csvLine.split("|")[102],
                                        "ClaimedNaturalCauses"	: WDB_Case_csvLine.split("|")[103],
                                        "Nodefence"				: WDB_Case_csvLine.split("|")[104],
                                        "DefenseNotes" 			: WDB_Case_csvLine.split("|")[105],
                                        "CaseNotes"		: WDB_Case_csvLine.split("|")[106],
                                        "Createdby"		: WDB_Case_csvLine.split("|")[107],
                                        "Createdate"	: WDB_Case_csvLine.split("|")[108],
                                        "Lastupdatedby"	: WDB_Case_csvLine.split("|")[109],
                                        "Lastupdatedon"	: WDB_Case_csvLine.split("|")[110]
                                        }
                } 
        }
        );

    if result.modified_count == 1:
        nmods = nmods + 1
    else:
        nnomods = nnomods + 1
    
    print "\r", i, "Modificados:", nmods, " No modificados:", nnomods,
    

WDB_Case_csvFile.close()

3413 Modificados: 3212  No modificados: 201


#### Hemos verificado con los datos de origen que hay 201 casos que no hacen referencia a ningún acusado. Esto causa que no se puedan cargar como subdocumentos al no encontrar el documento principal y que tenga sentido crear una colección independiente.

In [31]:
from pymongo import InsertOne, DeleteOne, ReplaceOne

# Eliminamos antes resultados de cargas anteriores
result = dbCnxSWP.WDB_Case.delete_many({})
print result.deleted_count

WDB_Case_csvFile = open(WDB_Case_data_path, "r")

iMax = 4096 # > 3413 -> cargará todos los registros
i = 1

for WDB_Case_csvLine in WDB_Case_csvFile:

    if i > iMax:
        break

    i = i + 1 

    result = dbCnxSWP.WDB_Case.insert_one(
            {
                						"_id"       	: WDB_Case_csvLine.split("|")[0],
                						"CaseRef"		: WDB_Case_csvLine.split("|")[0],
                                        "CaseId"		: WDB_Case_csvLine.split("|")[1],
                                        "CaseSystemId"		: WDB_Case_csvLine.split("|")[2],
                                        "NamedIndividual"	: WDB_Case_csvLine.split("|")[3],
                                        "AccusedRef"		: WDB_Case_csvLine.split("|")[4],
                                        "CaseStart_date"	: WDB_Case_csvLine.split("|")[5],
                                        "CaseStart_date_as_date"	: WDB_Case_csvLine.split("|")[6],
                                        "Case_date"					: WDB_Case_csvLine.split("|")[7],
                                        "Case_date_as_date"		: WDB_Case_csvLine.split("|")[8],
                                        "Age_at_case"			: WDB_Case_csvLine.split("|")[9],
                                        "CaseCommonName"	: WDB_Case_csvLine.split("|")[10],
                                        "Commission"		: WDB_Case_csvLine.split("|")[11],
                                        "Complaint"			: WDB_Case_csvLine.split("|")[12],
                                        "Correspondence"	: WDB_Case_csvLine.split("|")[13],
                                        "Chronicle"			: WDB_Case_csvLine.split("|")[14],
                                        "Other"				: WDB_Case_csvLine.split("|")[15],
                                        "Suspects_text"			: WDB_Case_csvLine.split("|")[16],
                                        "UNorthodoxRelPract_p"	: WDB_Case_csvLine.split("|")[17],
                                        "UNorthodoxRelPract_s"	: WDB_Case_csvLine.split("|")[18],
                                        "Consulting_p"			: WDB_Case_csvLine.split("|")[19],
                                        "Consulting_s"			: WDB_Case_csvLine.split("|")[20],
                                        "Demonic_p"			: WDB_Case_csvLine.split("|")[21],
                                        "Demonic_s"			: WDB_Case_csvLine.split("|")[22],
                                        "Demonic_possess_p"	: WDB_Case_csvLine.split("|")[23],
                                        "Demonic_possess_s"	: WDB_Case_csvLine.split("|")[24],
                                        "Fairies_p"			: WDB_Case_csvLine.split("|")[25],
                                        "Fairies_s"			: WDB_Case_csvLine.split("|")[26],
                                        "Folk_healing_p"	: WDB_Case_csvLine.split("|")[27],
                                        "Folk_healing_s"	: WDB_Case_csvLine.split("|")[28],
                                        "Maleficium_p"		: WDB_Case_csvLine.split("|")[29],
                                        "Maleficium_s"	: WDB_Case_csvLine.split("|")[30],
                                        "Midwifery_p"	: WDB_Case_csvLine.split("|")[31],
                                        "Midwifery_s"			: WDB_Case_csvLine.split("|")[32],
                                        "ImplicatedByAnother_p"	: WDB_Case_csvLine.split("|")[33],
                                        "ImplicatedByAnother_s"	: WDB_Case_csvLine.split("|")[34],
                                        "Neighbhd_dispute_p"	: WDB_Case_csvLine.split("|")[35],
                                        "Neighbhd_dispute_s"	: WDB_Case_csvLine.split("|")[36],
                                        "PoliticalMotive_p"		: WDB_Case_csvLine.split("|")[37],
                                        "PoliticalMotive_s"	: WDB_Case_csvLine.split("|")[38],
                                        "PropertyMotive_p"	: WDB_Case_csvLine.split("|")[39],
                                        "PropertyMotive_s"	: WDB_Case_csvLine.split("|")[40],
                                        "RefusedCharity_p"	: WDB_Case_csvLine.split("|")[41],
                                        "RefusedCharity_s"	: WDB_Case_csvLine.split("|")[42],
                                        "Treason_p"			: WDB_Case_csvLine.split("|")[43],
                                        "Treason_s"			: WDB_Case_csvLine.split("|")[44],
                                        "Other_p"			: WDB_Case_csvLine.split("|")[45],
                                        "Other_s"			: WDB_Case_csvLine.split("|")[46],
                                        "OtherText" 		: WDB_Case_csvLine.split("|")[47],
                                        "NotEnoughInfo_p"	: WDB_Case_csvLine.split("|")[48],
                                        "NotEnoughInfo_s"	: WDB_Case_csvLine.split("|")[49],
                                        "WhiteMagic_p"		: WDB_Case_csvLine.split("|")[50],
                                        "WhiteMagic_s"	: WDB_Case_csvLine.split("|")[51],
                                        "Charnotes"		: WDB_Case_csvLine.split("|")[52],
                                        "DemonicPact"	: WDB_Case_csvLine.split("|")[53],
                                        "DevilNotes"		: WDB_Case_csvLine.split("|")[54],
                                        "WitchesMeeting"	: WDB_Case_csvLine.split("|")[55],
                                        "MeetingName"		: WDB_Case_csvLine.split("|")[56],
                                        "DevilPresent"		: WDB_Case_csvLine.split("|")[57],
                                        "Maleficium"	: WDB_Case_csvLine.split("|")[58],
                                        "CommunalSex"	: WDB_Case_csvLine.split("|")[59],
                                        "DevilWorship"	: WDB_Case_csvLine.split("|")[60],
                                        "FoodAndDrink"	: WDB_Case_csvLine.split("|")[61],
                                        "Dancing"		: WDB_Case_csvLine.split("|")[62],
                                        "Singing"		: WDB_Case_csvLine.split("|")[63],
                                        "SingingText" 		: WDB_Case_csvLine.split("|")[64],
                                        "OtherPractices" 	: WDB_Case_csvLine.split("|")[65],
                                        "MeetingNotes"		: WDB_Case_csvLine.split("|")[66],
                                        "Elphane/Fairyland"	: WDB_Case_csvLine.split("|")[67],
                                        "Food/Drink"				: WDB_Case_csvLine.split("|")[68],
                                        "SpecificVerbalFormulae"	: WDB_Case_csvLine.split("|")[69],
                                        "SpecificRitualActs"		: WDB_Case_csvLine.split("|")[70],
                                        "Familiars"					: WDB_Case_csvLine.split("|")[71],
                                        "Shape-Changing"			: WDB_Case_csvLine.split("|")[72],
                                        "Dreams/Visions"				: WDB_Case_csvLine.split("|")[73],
                                        "UnorthodoxReligiousPractice"	: WDB_Case_csvLine.split("|")[74],
                                        "SympatheticMagic"				: WDB_Case_csvLine.split("|")[75],
                                        "Ridingdead"		: WDB_Case_csvLine.split("|")[76],
                                        "FolkNotes"			: WDB_Case_csvLine.split("|")[77],
                                        "HumanIllness"		: WDB_Case_csvLine.split("|")[78],
                                        "HumanDeath"		: WDB_Case_csvLine.split("|")[79],
                                        "AnimalIllness"		: WDB_Case_csvLine.split("|")[80],
                                        "AnimalDeath"		: WDB_Case_csvLine.split("|")[81],
                                        "FemaleInfertility"	: WDB_Case_csvLine.split("|")[82],
                                        "MaleImpotence"		: WDB_Case_csvLine.split("|")[83],
                                        "AggravatingDisease"	: WDB_Case_csvLine.split("|")[84],
                                        "TransferringDisease"	: WDB_Case_csvLine.split("|")[85],
                                        "LayingOn"				: WDB_Case_csvLine.split("|")[86],
                                        "Removalbewitchment"	: WDB_Case_csvLine.split("|")[87],
                                        "Quarreling"		: WDB_Case_csvLine.split("|")[88],
                                        "Cursing"			: WDB_Case_csvLine.split("|")[89],
                                        "Poisoning"			: WDB_Case_csvLine.split("|")[90],
                                        "RecHealer"			: WDB_Case_csvLine.split("|")[91],
                                        "HealingHumans"		: WDB_Case_csvLine.split("|")[92],
                                        "HealingAnimals"	: WDB_Case_csvLine.split("|")[93],
                                        "Midwifery"			: WDB_Case_csvLine.split("|")[94],
                                        "DiseaseNotes" 		: WDB_Case_csvLine.split("|")[95],
                                        "PropertyDamage"		: WDB_Case_csvLine.split("|")[96],
                                        "WeatherModification"	: WDB_Case_csvLine.split("|")[97],
                                        "OtherMaleficiaNotes" 	: WDB_Case_csvLine.split("|")[98],
                                        "OtherChargesNotes" 	: WDB_Case_csvLine.split("|")[99],
                                        "ClaimedBewitched"		: WDB_Case_csvLine.split("|")[100],
                                        "ClaimedPossessed"		: WDB_Case_csvLine.split("|")[101],
                                        "AdmitLesserCharge"		: WDB_Case_csvLine.split("|")[102],
                                        "ClaimedNaturalCauses"	: WDB_Case_csvLine.split("|")[103],
                                        "Nodefence"				: WDB_Case_csvLine.split("|")[104],
                                        "DefenseNotes" 			: WDB_Case_csvLine.split("|")[105],
                                        "CaseNotes"		: WDB_Case_csvLine.split("|")[106],
                                        "Createdby"		: WDB_Case_csvLine.split("|")[107],
                                        "Createdate"	: WDB_Case_csvLine.split("|")[108],
                                        "Lastupdatedby"	: WDB_Case_csvLine.split("|")[109],
                                        "Lastupdatedon"	: WDB_Case_csvLine.split("|")[110]
            }
        )
    print "\rresult.inserted_id:", result.inserted_id, i, 
    
    
WDB_Case_csvFile.close()


0
result.inserted_id: C/LA/3406 3414


### Los 10 años más "*prolíficos*"

In [32]:
csr = dbCnxSWP.WDB_Case.aggregate([
  { "$project" : { "arrCase_date_as_date" : { "$split" : ["$Case_date_as_date" , " "] } } },
  { "$project" : { "date" : { "$arrayElemAt" : ["$arrCase_date_as_date" , 0] } } },
  { "$project" : { "arrDate" : { "$split" : ["$date" , "/"] } } },
  { "$project" : { "year" : { "$arrayElemAt" : ["$arrDate" , 2] } } },
  { "$group" : { "_id" : "$year" , "Casos" : { "$sum" : 1 } } },
  { "$sort" : { "Casos" : -1 } },
  { "$limit" : 10}
]);
for item in csr:
    print item["Casos"], "casos en", item["_id"]+ "."

407 casos en 1662.
399 casos en 1649.
240 casos en 1661.
213 casos en 1650.
183 casos en 1629.
121 casos en 1597.
116 casos en 1643.
114 casos en 1630.
86 casos en 1659.
85 casos en 1591.


#  
## A continuación el diagrama de las entidades y atributos utilizados en estas consultas

<img src="http://localhost:8001/files/SWP-ScotWitchProject/images/MongoDB.png">

### Conclusiones:
* Considero que MongoDB presenta ventajas en este caso para el tratamiento de información no estructurada: notas de las acusaciones, textos y campos no categorizados.
######  
* También permitiría un acceso más eficiente en desarrollo a la información, dado el elevado número de relaciones y datos ausentes. 
######  
* Por supuesto,también permitiría la inclusiónde información no estructurada. En lugar de la referencia a las fuentes (documentos de bibliotecas, parroquias, universidades, etc...) se podrían incluir los documentos y realizar consultas que pudiesen derivar en nuevas relaciones.  
######  
* Por último, podrían añadirse herramientas de *web scraping* que permitieran ampliar la información no estructurada registrada. Podrían detectarse en búsquedas la aparición de nuevos datos sobre los casos derivados de otros proyectos de investigación que se divulgen en Internet.